<div style="border-radius:16px;background:#1e2a1e;margin:1em 0;padding:1em 1em 1em 3em;color:#eceff4;position:relative;box-shadow:0 6px 16px rgba(0,0,0,.4)">
  <b style="color:#a3be8c;font-size:1.25em">Your Challenge:</b>
  <ul style="margin:.6em 0 0;padding-left:1.2em;line-height:1.6">
    <li>Hey everyone! Ready to flex those agentic muscles? 🎉 Build a workflow just like the ticket system above, but for <b>product reviews</b>!</li>
    <li>Your workflow should:
      <ul>
        <li>Generate a product review (think: electronics, books, or your favorite kitchen gadget)</li>
        <li>Respond to the review (company reply, moderation, or a witty bot response)</li>
        <li>Evaluate the response (is it helpful, polite, and on point?)</li>
      </ul>
    </li>
    <li>Use structured outputs and Pydantic models for each step, just like we did above.</li>
    <li>Include an evaluator step to assess the quality of the response.</li>
    <li>Here’s a suggested workflow to get your creative gears turning:</li>
  </ul>
  <div style="position:absolute;top:-.8em;left:-.8em;width:2.4em;height:2.4em;border-radius:50%;background:#a3be8c;color:#2e3440;display:flex;align-items:center;justify-content:center;font-weight:700;font-size:1.2em">💪</div>
</div>

In [1]:
import os
from openai import OpenAI
from dotenv import load_dotenv
from IPython.display import display, Markdown

load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

if not OPENAI_API_KEY:
    raise ValueError("Please set the OPENAI_API_KEY environment variable.")
    

In [2]:
from pydantic import BaseModel

class ProductReview(BaseModel):
    product_id: str
    product_name: str
    review_text: str
    rating: int

class ReviewResponse(BaseModel):
    review_sentiment: str
    company_response: str
    

class ResponseEvaluation(BaseModel):
    is_helpful: bool
    is_accurate: bool
    is_polite: bool
    overall_feedback: str
    


In [3]:
client = OpenAI()

In [4]:
# messages list
user_message = "I want you to generate a product review. "
user_message += "The review should be a single sentence describing the user's experience with the product. "
user_message += "Please ensure the review is varied and covers different types of experiences."

messages = [{"role": "user", "content": user_message}]

In [8]:
# use .parse instead of .create() to get structured response
response = client.chat.completions.parse(
    model="gpt-4.1-nano",
    messages=messages,
    response_format=ProductReview
)
structured_response = response.choices[0].message.parsed
display(Markdown(f"**Generated Product Review:** {structured_response}"))

**Generated Product Review:** product_id='XYZ123' product_name='SuperComfort Running Shoes' review_text='These shoes provided excellent support and comfort during my long runs, making them a great addition to my workout gear.' rating=5

In [9]:
structured_response.review_text

'These shoes provided excellent support and comfort during my long runs, making them a great addition to my workout gear.'

## Respond to review


In [12]:
# messages list
message = "You are to respond to the review as the seller. review_sentiment should be classified as positive, neutral or negative. company_response should be a short paragraph polite response to the review.\n\n"
message += f"Review: {structured_response.review_text}\n"
message += f"Product name: {structured_response.product_name}\n"
message += f"Review rating: {structured_response.rating}\n"

messages = [{"role": "user", "content": message}]


In [13]:
company_response = client.chat.completions.parse(
    model="gpt-4.1-nano",
    messages=messages,
    response_format=ReviewResponse
)
structured_company_response = company_response.choices[0].message.parsed
display(Markdown(f"**Company Response:** {structured_company_response}"))

**Company Response:** review_sentiment='positive' company_response="Thank you for your kind words! We're thrilled to hear that our SuperComfort Running Shoes have enhanced your running experience. Your satisfaction is our priority, and we look forward to supporting your fitness journey."

## Evaluate the response

In [14]:
# messages list
message = "You are to evaluate the company response for the following customer review. "
message += "You will determine if the response is appropriate for the review. "
message += "\n\n"
message += f"Review: {structured_response.review_text}\n"
message += f"Product name: {structured_response.product_name}\n"
message += f"Review rating: {structured_response.rating}\n\n"
message += f"Company response: {structured_company_response.company_response}\n"
message += f"Sentiment classification: {structured_company_response.review_sentiment}\n\n"

messages = [{"role": "user", "content": message}]


In [15]:
evaluation_response = client.chat.completions.parse(
    model="gpt-4.1-nano",
    messages=messages,
    response_format=ResponseEvaluation
)
structured_evaluation_response = evaluation_response.choices[0].message.parsed
display(Markdown(f"**Evaluation Response:** {structured_evaluation_response}"))

**Evaluation Response:** is_helpful=True is_accurate=True is_polite=True overall_feedback="The company's response is appropriate for the customer's positive review. It acknowledges the customer's satisfaction, expresses appreciation, and encourages continued engagement, which are all suitable responses to a positive review."